<h1> DattoRMM - Devices - Data Ingestion - MongoDB</h1>

# Import Modules

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import json


# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# DattoRMM - Pull JSON from API

## Create Auth Token

### Create DataFrame via API Call Iteration


In [ ]:
# DattoRMM DataFrame - import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']
# call token api url
token_uri = config['dattormm']['token_uri']

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']
## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

# Data Shaping

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_devices.set_index('uid',inplace=True)

### Type | Category

In [ ]:
def device_category(device):
    if device is None:
        return None
    else:
        return device['category']

In [ ]:
def device_type(device):
    if device is None:
        return None
    else:
        return device['type']

In [ ]:
df_devices['category'] = df_devices['deviceType'].apply(device_category)
df_devices['type'] = df_devices['deviceType'].apply(device_type)

# Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_devices['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
df_devices.drop(columns='deviceType',inplace=True)

### Patch Management Breakdown
 patchStatus | patchesApprovedPending | patchesNotApproved | patchesInstalled

In [ ]:
# patchStatus
def patch_status(patch_management):
    return patch_management['patchStatus']

df_devices['patchStatus'] = df_devices['patchManagement'].apply(patch_status)

# patchesApprovedPending
def patches_approved_pending(patch_management):
    return patch_management['patchesApprovedPending']

df_devices['patchesApprovedPending'] = df_devices['patchManagement'].apply(patches_approved_pending)

# patchesNotApproved
def patches_not_approved(patch_management):
    return patch_management['patchesNotApproved']

df_devices['patchesNotApproved'] = df_devices['patchManagement'].apply(patches_not_approved)

# patchesInstalled
def patches_installed(patch_management):
    return patch_management['patchesInstalled']

df_devices['patchesInstalled'] = df_devices['patchManagement'].apply(patches_installed)


# drop patchManagement {inplace=True}
df_devices.drop('patchManagement',axis=1,inplace=True)

## Drop 'antivirus'

In [ ]:
df_devices.drop('antivirus',axis=1,inplace=True)

## Create Time Columns and Timedate Shaping

### Add Timezone Column from UDF

In [ ]:
# Timezone
def local_timezone(udf):
    return udf['udf10']

df_devices['localTimezone'] = df_devices['udf'].apply(local_timezone)

# drop udf {inplace=True}
df_devices.drop('udf',axis=1,inplace=True)

### Create Date Correlation Columns

In [ ]:
# all date columns
parse_dates =  ['lastAuditDate','lastSeen','lastReboot','creationDate',]

### Convert Epoch to UTC

In [ ]:
df_devices['lastAuditDate'] = pd.to_datetime(df_devices['lastAuditDate'],unit='ms',errors='coerce')
#df_devices['lastAuditDate'].head(5)

In [ ]:
df_devices['lastSeen'] = pd.to_datetime(df_devices['lastSeen'],unit='ms',errors='coerce')
#df_devices['lastSeen'].head(5)

In [ ]:
df_devices['creationDate'] = pd.to_datetime(df_devices['creationDate'],unit='ms',errors='coerce')
#df_devices['creationDate'].head(5)

In [ ]:
df_devices['lastReboot'] = pd.to_datetime(df_devices['lastReboot'],unit='ms',errors='coerce')
#df_devices['lastReboot'].head(5)

### Define and apply functions to create correlation columns

In [ ]:
def no_audit_7_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=7):
        return 1
    else:
        return 0

In [ ]:
def offline_30_days(last_seen):
    if last_seen < dt.datetime.now() - dt.timedelta(days=60):
        return 1
    else:
        return 0

In [ ]:
def no_reboot_60_days(last_reboot):
    if last_reboot < dt.datetime.now() - dt.timedelta(days=60):
        return 1
    else:
        return 0

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit7Days'] = df_devices['lastAuditDate'].apply(no_audit_7_days)

In [ ]:
# Create Column - Devices Offline 30 Days
df_devices['offline30Days'] = df_devices['lastSeen'].apply(offline_30_days)

In [ ]:
# Create Column - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_devices['noReboot60Days'] = df_devices['lastReboot'].apply(no_reboot_60_days)

In [ ]:
# Create DF copy for reference
df_raw_data = df_devices

## DattoRMM DataFrame Data Standardization Shaping

### Hostname to_upper()

In [ ]:
df_devices['hostname'] = df_devices['hostname'].str.upper()

### Replace Values {'TRUE':1,'FALSE':0}

In [ ]:
df_devices.replace({True:1,False:0},inplace=True)

In [ ]:
convert_to_int = df_devices.dtypes[(df_devices.dtypes == 'float') | (df_devices.dtypes == 'bool') | (df_devices.dtypes == 'uint8')].index.tolist()

In [ ]:
df_devices[convert_to_int] = df_devices[convert_to_int].astype('Int64')

### Drop Columns with no Data (NaN, 0, or None as only Value on all rows)

In [ ]:
df_devices.drop(['lastLoggedInUser','warrantyDate'],axis=1,inplace=True)

## Add 'patchStatus' Dummy Columns

In [ ]:
df_patch_status = pd.get_dummies(df_devices['patchStatus'],prefix='patchStatus')
df_patch_status.drop('patchStatus_NoPolicy',axis=1, inplace=True)
df_devices = df_devices.join(df_patch_status)
df_devices.drop('patchStatus',axis=1,inplace=True)

# MongoDB

In [ ]:
username = config['mongodb']['username']
password = config['mongodb']['password']
connection_ip = config['mongodb']['connection_ip']
database = 'seed_data'

# import bson for object encoding
import bson

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = f"mongodb://{username}:{password}@{connection_ip}/{database}"
CONNECTION_STRING = 'mongodb://localhost:27017'


# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
from pymongo import MongoClient
client = MongoClient(CONNECTION_STRING)


db = client['datto_rmm']
collection = db['seed_data']


In [ ]:
df_ingest = df_devices.reset_index()

In [ ]:
dt_list = list(df_ingest.dtypes[df_ingest.dtypes == 'datetime64[ns]'].index)

In [ ]:
df_ingest.fillna(pd.NA,inplace=True)

In [ ]:
entries = []
cols = enumerate(list(df_ingest.columns))

for index, row in df_ingest.iterrows():
    current_row = row.to_dict()
    new_entry = {}
    try:
        try:
            new_entry['onlineData'] = {'legend':['onlineStatus','lastSeen'],'data':[[current_row['online'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "onlineData"')
        try:
            if current_row['suspended'] == 1:
                new_entry['isSuspended'] = {'legend':['suspendedStatus','lastSeen'],'data':[[current_row['suspended'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "isSuspended"')
            print(current_row['suspended'])
        try:
            if current_row['deleted'] == 1:
                new_entry['isDeleted'] = {'legend':['deletedStatus','lastSeen'],'data':[[current_row['deleted'],current_row['lastSeen']]]}
        except:
            print(f'Unable to create column: "isDeleted"')
            print(current_row['deleted'])
        try:
            if current_row['rebootRequired'] == 1:
                new_entry['rebootRequired'] = {'legend':['rebootRequired','lastReboot'],'data':[[current_row['rebootRequired'],current_row['lastReboot']]]}
        except:
            print(f'Unable to create column: "rebootRequired"')
            print(current_row['rebootRequired'])
        for k,v in current_row.items():
            if pd.isna(v) == True:
                continue
            elif (k == 'online') | (k == 'lastSeen') | (k == 'suspended') | (k == 'rebootRequired') | (k == 'lastReboot') |(k == 'deleted'):
                continue
            else:
                new_entry[k] = v

        entries.append(new_entry)
    except:
        print(current_row)

In [ ]:


for entry in entries:
    try:
        result = collection.find_one({'uid':entry['uid']})
        print(f"_id found: {result['_id']}")
        match_id = result['_id']
        for k,v in entry.items():
            try:
                #print(k,v)
                if k in  ['onlineData','isSuspended','isDeleted','rebootRequired']:
                    try:
                        #print({f"{ str(k)  + '.legend'}": v['legend']})
                        #collection.insert_one({'_id':m_id}, {f"{str(k)}":v['legend']})
                        collection.update_one({'_id':match_id},{'$set': {f"{ str(k)  + '.legend'}" : v['legend'] }},upsert=False)
                        collection.update_one({'_id':match_id},{'$push': {f"{ str(k)  + '.data'}" : {"$each":  v['data'] }}},upsert=True)
                        #print(f"Successfully updated {k} for {entry['hostname']}")
                    except Exception as error:
                        print(error)
                else:
                    try:
                        collection.update_one({'_id':match_id}, {'$set':{f"{str(k)}":v}},upsert=True)
                        #print(f"Successfully inserted new {k} for {entries[0]['hostname']}")
                    except Exception as error:
                        #print(f"Unable to insert new {k} for {entries[0]['hostname']}")
                        print(error)
            except Exception as e:
                print(e)
                continue
    except Exception as e:
        try:
            print(f"_id not found: attempting new entry for {entry['hostname']} with uid: {entry['uid']}")
            collection.insert_one(entry)
        except:
            print(e)

print('*'*120)
print('End of Import!')